In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import time
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
import selenium
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import undetected_chromedriver as uc
import subprocess
import requests
import re
import numpy as np
import requests
import datetime

In [4]:
from sqlalchemy import create_engine
import urllib

In [5]:
quoted = urllib.parse.quote_plus("Driver={SQL Server Native Client 11.0};Server=VIKTOR-PC-2001;Database=DividendPolicyResearch;Trusted_Connection=yes;")
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))
result = engine.execute('SELECT [Ticker] FROM [dbo].[tb_sp100_tickers]')

In [6]:
df_SP100_companies = pd.DataFrame(result.fetchall())
tickers_list = df_SP100_companies.iloc[:, 0].tolist()
tickers_list

['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'ADBE',
 'AIG',
 'AMD',
 'AMGN',
 'AMT',
 'AMZN',
 'AVGO',
 'AXP',
 'BA',
 'BAC',
 'BK',
 'BKNG',
 'BLK',
 'BMY',
 'BRK-B',
 'C',
 'CAT',
 'CHTR',
 'CL',
 'CMCSA',
 'COF',
 'COP',
 'COST',
 'CRM',
 'CSCO',
 'CVS',
 'CVX',
 'DE',
 'DHR',
 'DIS',
 'DOW',
 'DUK',
 'EMR',
 'EXC',
 'F',
 'FDX',
 'GD',
 'GE',
 'GILD',
 'GM',
 'GOOG',
 'GOOGL',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KHC',
 'KO',
 'LIN',
 'LLY',
 'LMT',
 'LOW',
 'MA',
 'MCD',
 'MDLZ',
 'MDT',
 'MET',
 'META',
 'MMM',
 'MO',
 'MRK',
 'MS',
 'MSFT',
 'NEE',
 'NFLX',
 'NKE',
 'NVDA',
 'ORCL',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTX',
 'SBUX',
 'SCHW',
 'SO',
 'SPG',
 'T',
 'TGT',
 'TMO',
 'TMUS',
 'TSLA',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB',
 'V',
 'VZ',
 'WFC',
 'WMT',
 'XOM']

In [7]:
print(len(tickers_list))

101


In [8]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
driver = uc.Chrome(options=options)
driver.maximize_window()

for ticker in tickers_list:
    ticker = ticker.replace("-",".")
    print("Тікер: " + ticker)

    wait = WebDriverWait(driver, 20)
    urlnm = 'https://stockanalysis.com/stocks/'+ticker+'/company/'
    driver.get(urlnm)
    driver.implicitly_wait(1)
      
    cpnm = driver.find_element(By.CSS_SELECTOR, "div.text-center:nth-child(1)").text
    cpnm = cpnm.lower()
    print("Назва компанії: " + cpnm)
    
    cpnm = cpnm.split()
    if cpnm[0].replace(".","").replace(",","") == 'the':
        cpnm = cpnm[1].replace(".","").replace(",","")
    else:
        cpnm = cpnm[0].replace(".","").replace(",","")
    print("Скорочена назва компанії: " + cpnm)
    
    url_income_statement = 'https://www.macrotrends.net/stocks/charts/'+ticker+'/'+cpnm+'/income-statement?freq=A'
    print("Посилання: " + url_income_statement)
    time.sleep(2)
    
    #driver = webdriver.Chrome()
    #driver.maximize_window()
    driver.execute_script(
        f"window.location = '{url_income_statement}';")
    driver.implicitly_wait(2)

    # частина 1
    grid = driver.find_element(By.ID, 'wrapperjqxgrid')
    time.sleep(1)

    soup = BeautifulSoup(grid.get_attribute('outerHTML'), "html.parser")
    headersList = soup.findAll('div', {'role': 'columnheader'})
    col_names=[h.text for h in headersList]
    divList = soup.findAll('div', {'role': 'row'})
    data = [[x.text for x in div.findChildren('div', recursive=False)] for div in divList]
    df = pd.DataFrame(data, columns=col_names)
    print("К-ть стовпців (частина 1): " + str(df.index.size))
    
    df = df.rename(columns={"Annual Data | Millions of US $ except per share data": "Year"})
    df = df.transpose()
    df['index1'] = df.index
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[2:], columns=headers)
    print("К-ть рядків (частина 1): " + str(new_df.index.size))
    new_df['ticker_name'] = ticker.replace(".","-")
    new_df["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
    new_df["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
    new_df["period type"] = 1
    
    new_df.to_sql('tb_FS_income_statement_macrotrends', schema='dbo', con = engine, if_exists='append',index=False)
    
    driver.implicitly_wait(2)

    # частина 2
    grid = driver.find_element(By.ID, 'wrapperjqxgrid')
    time.sleep(1)
    driver.execute_script("window.scrollBy(0, 600);")
    scrollbar = driver.find_element(By.ID, 'jqxScrollThumbhorizontalScrollBarjqxgrid')

    time.sleep(1)

    actions = ActionChains(driver)
    time.sleep(1)

    for i in range(1, 6):
        actions.drag_and_drop_by_offset(scrollbar, i * 70, 0).perform()
    time.sleep(1)
    
    soup = BeautifulSoup(grid.get_attribute('outerHTML'), "html.parser")
    headersList = soup.findAll('div', {'role': 'columnheader'})
    col_names=[h.text for h in headersList]
    divList = soup.findAll('div', {'role': 'row'})
    data = [[x.text for x in div.findChildren('div', recursive=False)] for div in divList]
    df = pd.DataFrame(data, columns=col_names)
    print("К-ть стовпців (частина 2): " + str(df.index.size))
    
    df = df.rename(columns={"Annual Data | Millions of US $ except per share data": "Year"})
    df = df.transpose()
    df['index1'] = df.index
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[2:], columns=headers)
    print("К-ть рядків (частина 2): " + str(new_df.index.size))
    new_df['ticker_name'] = ticker.replace(".","-")
    new_df["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
    new_df["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
    new_df["period type"] = 2
    
    new_df.to_sql('tb_FS_income_statement_macrotrends', schema='dbo', con = engine, if_exists='append',index=False)
    
    print("")

Тікер: AAPL
Назва компанії: apple inc
Скорочена назва компанії: apple
Посилання: https://www.macrotrends.net/stocks/charts/AAPL/apple/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 7

Тікер: ABBV
Назва компанії: abbvie inc.
Скорочена назва компанії: abbvie
Посилання: https://www.macrotrends.net/stocks/charts/ABBV/abbvie/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 7

Тікер: ABT
Назва компанії: abbott laboratories
Скорочена назва компанії: abbott
Посилання: https://www.macrotrends.net/stocks/charts/ABT/abbott/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 7

Тікер: ACN
Назва компанії: accenture plc
Скорочена назва компанії: accenture
Посилання: https://www.macrotrends.net/stocks/charts/ACN/accenture/income-statement?freq=A

JavascriptException: Message: javascript error: Unexpected identifier 's'
  (Session info: chrome=123.0.6312.58)
Stacktrace:
	GetHandleVerifier [0x01474CE3+225091]
	(No symbol) [0x013A4E31]
	(No symbol) [0x01249A7A]
	(No symbol) [0x0124DEB0]
	(No symbol) [0x0124FA76]
	(No symbol) [0x012B65E2]
	(No symbol) [0x0129F55C]
	(No symbol) [0x012B5CB0]
	(No symbol) [0x0129F2F6]
	(No symbol) [0x012779B9]
	(No symbol) [0x0127879D]
	sqlite3_dbdata_init [0x018E9A83+4064547]
	sqlite3_dbdata_init [0x018F108A+4094762]
	sqlite3_dbdata_init [0x018EB988+4072488]
	sqlite3_dbdata_init [0x015EC9E9+930953]
	(No symbol) [0x013B0804]
	(No symbol) [0x013AAD28]
	(No symbol) [0x013AAE51]
	(No symbol) [0x0139CAC0]
	BaseThreadInitThunk [0x774462C4+36]
	RtlSubscribeWnfStateChangeNotification [0x77DA0FA9+1081]
	RtlSubscribeWnfStateChangeNotification [0x77DA0F74+1028]
